## Prerequisites
This notebook requires an experimental version of the cmdty-storage package only published on Test PyPI, plus the package ipysheet which provides the grids.

`
pip install -i https://test.pypi.org/simple/ cmdty-storage==0.1.0a15
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()

### To Do
* Sort out truncation of labels.
* Interest rate curve, not flat rate.
* Ability to select granularity.
* Logging.
* Display more output data as tables using qgrid:
    * Trigger price.
    * Projected inventory.
    * Simulation results.
    

In [2]:
sg.test_data_btn()

Button(description='Populate Test Data', style=ButtonStyle())